<a href="https://colab.research.google.com/github/krishna-krish3/Covid-Detection/blob/main/covid_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from matplotlib import pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms, models
from PIL import Image
import numpy as np
import os
from torch.utils.data.sampler import SubsetRandomSampler
import pandas as pd

In [ ]:
#from cadence models
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
from torchvision import datasets, transforms
import helper
import matplotlib.pyplot as plt
transform = transforms.Compose([transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.48, 0.45, 0.40], std=[0.23, 0.23, 0.23])])
                                
dataset = datasets.ImageFolder("/content/gdrive/MyDrive/Colab Notebooks/Covid_Images/", transform = transform)
bsize = len(dataset)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=bsize, shuffle=False, pin_memory=True)
images, labels = next(iter(dataloader))
images, labels = images.cuda(), labels.cuda()
#helper.imshow(images[0], normalize=False)

In [ ]:
len(dataset)

7330

In [ ]:
#cifar10...32x32
num_classes=len(torch.unique(labels))
num_classes

1

In [ ]:
trainlen=round(.8*len(dataset))
vallen=round(0.1*len(dataset))
testlen=len(dataset)-round(trainlen+vallen)
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [trainlen, vallen, testlen], generator=torch.Generator().manual_seed(42))

In [ ]:
len(trainlen)

In [ ]:
len(testlen)

In [ ]:
len(vallen)

In [ ]:
from torch.utils.data import  DataLoader
train_loader = DataLoader(train_set, batch_size=8, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=8, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=8, shuffle=True, num_workers=2)

In [ ]:
len(train_set)

In [ ]:
len(test_set)

In [ ]:
len(val_set)

In [ ]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torchvision import transforms, datasets, models, utils
from torch.utils.tensorboard import SummaryWriter
import time
import numpy as np
from torchsummary import summary
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
# Get a batch of training data
inputs, classes = next(iter(train_loader))
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
imshow(out)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# #pretrained=True will download a pretrained network for us
# model = models.densenet201(pretrained=True)
# model

In [ ]:
###Rwightman Models
# !pip install timm
# import timm
# import torch
# from torch.autograd import Variable
# model = timm.create_model('resnet18', pretrained=True)
# model 


In [ ]:
#timm.list_models()

In [ ]:
!pip install pytorchcv torch>=0.4.0
from pytorchcv.model_provider import get_model as ptcv_get_model
import torch
from torch.autograd import Variable

model = ptcv_get_model("regnetx002", pretrained=True)
model

In [ ]:
#Freezing model parameters and defining the fully connected network to be attached to the model, loss function and the optimizer.
#We there after put the model on the GPUs
#num_ftrs = model.output.fc3.in_features
#num_ftrs = model.output.fc.in_features
num_ftrs = 368
#num_ftrs = model.output.in_features
for param in model.parameters():
  param.require_grad = False
fc = nn.Sequential(
    # nn.Linear(1920, 1920),
    # nn.ReLU(),
    # nn.Dropout(0.4),
    
    nn.Linear(num_ftrs,num_classes),
    #nn.Dropout(0.6),
    #nn.LogSoftmax(dim=1)
    
)
model.output = fc
criterion = nn.CrossEntropyLoss()
#Over here we want to only update the parameters of the classifier so
optimizer = torch.optim.SGD(model.output.parameters(), lr=0.003)
#optimizer = torch.optim.Adam(model.output.parameters(), lr = 0.003)
model.to(device)

In [ ]:
num_epochs = 15
valid_loader = val_loader
# keeping-track-of-losses 
train_losses = []
valid_losses = []
train_acc = []
val_acc = []

for epoch in range(1, num_epochs + 1):
    # keep-track-of-training-and-validation-loss
    train_loss = 0.0
    valid_loss = 0.0
    correct = 0
    total=0
    # training-the-model
    model.train()
    for data, target in train_loader:
        # move-tensors-to-GPU 
        data = data.to(device)
        target = target.to(device)
        
        # clear-the-gradients-of-all-optimized-variables
        optimizer.zero_grad()
        # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
        output = model(data)
        # calculate-the-batch-loss
        loss = criterion(output, target)
        # backward-pass: compute-gradient-of-the-loss-wrt-model-parameters
        loss.backward()
        # perform-a-ingle-optimization-step (parameter-update)
        optimizer.step()
        # update-training-loss
        train_loss += loss.item() * data.size(0)
        ##TODAY
        _,pred = torch.max(output, dim=1)
        correct += torch.sum(pred==target).item()
        total += target.size(0)
    train_acc.append(100 * correct / total)
    # validate-the-model
    model.eval()
    total_t=0
    correct_t=0
    for data, target in valid_loader:
        
        data = data.to(device)
        target = target.to(device)
        
        output = model(data)
        
        loss = criterion(output, target)
        
        # update-average-validation-loss 
        valid_loss += loss.item() * data.size(0)
        ###today
        _,pred_t = torch.max(output, dim=1)
        correct_t += torch.sum(pred_t==target).item()
        total_t += target.size(0)
    val_acc.append(100 * correct_t / total_t)
    # calculate-average-losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
        
    # print-training/validation-statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='Validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(frameon=False)

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(frameon=False)

In [ ]:
# test-the-model
model.eval()  # it-disables-dropout
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

# Save 
torch.save(model.state_dict(), 'model.ckpt')

In [ ]:
# def train_model(train_dl, model):
#     # define the optimization
#     criterion = nn.CrossEntropyLoss()
# #Over here we want to only update the parameters of the classifier so
#     optimizer = torch.optim.SGD(model.output.parameters(), lr=0.003)
#     # enumerate epochs
#     losses = []
#     running_corrects = 0
#     epochacc = []
#     for epoch in range(10):
#         # enumerate mini batches
#         running_loss = 0.0
#         for i, (inputs, targets) in enumerate(train_dl):
#             inputs, targets = inputs.to(device), targets.to(device)
#             # clear the gradients
#             optimizer.zero_grad()
#             # compute the model output
#             yhat = model(inputs)
#             # calculate loss
#             loss = criterion(yhat, targets)
#             _, preds = torch.max(yhat, 1)
#             # credit assignment
#             loss.backward()
#             # update model weights
#             optimizer.step()
#             running_loss += loss.item() * inputs.size(0)
#             running_corrects += torch.sum(preds == targets.data)
#         epoch_loss = running_loss / len(train_loader)
#         epoch_acc = running_corrects.double() / len(train_loader)
#         losses.append(epoch_loss)
#         epoch_acc.tolist()
#         epochacc.append(epoch_acc)
#         running_corrects.append(epoch_acc)
#         print(epoch_acc) 
#     return losses

In [ ]:
# from numpy import argmax
# from numpy import vstack
# from sklearn.metrics import accuracy_score
# def evaluate_model(test_dl, model):
#     predictions, actuals = list(), list()
#     for i, (inputs, targets) in enumerate(test_dl):
#         inputs, targets = inputs.to(device), targets.to(device)
#         # evaluate the model on the test set
#         yhat = model(inputs)
#         # retrieve numpy array
#         yhat = yhat.cpu().detach().numpy()
#         actual = targets.cpu().numpy()
#         # convert to class labels
#         yhat = argmax(yhat, axis=1)
#         # reshape for stacking
#         actual = actual.reshape((len(actual), 1))
#         yhat = yhat.reshape((len(yhat), 1))
#         # store
#         predictions.append(yhat)
#         actuals.append(actual)
#     predictions, actuals = vstack(predictions), vstack(actuals)
#     # calculate accuracy
#     acc = accuracy_score(actuals, predictions)
#     return acc

In [ ]:
#losses = train_model(train_loader, model)
# import matplotlib.pyplot as plt
# plt.plot(losses)

In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(losses, label='TrainLoss', marker='o')
# #plt.plot(epochacc, label='TrainAcc', marker='x')
#   # plt.plot(x, history['val_loss'], label='val', marker='o')
#     # plt.plot(x, history['train_loss'], label='train', marker='o')
# plt.title('Loss per epoch'); plt.ylabel('Loss');
# plt.xlabel('Epoch')
# plt.legend(), plt.grid()
# plt.show()
    

In [ ]:
# trainacc = evaluate_model(train_loader, model)
# print('TrainAccuracy: %.3f' % trainacc)
# valacc = evaluate_model(val_loader, model)
# print('ValAccuracy: %.3f' % valacc)
# testacc = evaluate_model(test_loader, model)
# print('Accuracy: %.3f' % testacc)
